In [36]:
import pandas as pd
import nltk
from nltk import word_tokenize, pos_tag
from gensim import models
from gensim.corpora import Dictionary, MmCorpus
import re, string

In [ ]:
# import import_ipynb
# import preprocess.ipynb

importing Jupyter notebook from preprocess.ipynb


In [ ]:
# df_text

In [3]:
import ast
text_file = open("Datasets/anarchism_sentences.txt", "r")
df = pd.DataFrame(ast.literal_eval(text_file.read()), columns=['sentences'])
df['idx'] = df.index
df

,sentences,idx
0,'''Anarchism''' is an and that r...,0
1,"These institutions are often described as , a...",1
2,Anarchism's central disagreement with other id...,2
3,"Anarchism is usually placed on the of the ,...",3
4,As anarchism does not offer a fixed body of do...,4
...,...,...
268,"It also states that ""some sort of 'court' syst...",268
269,=== Industrial civilisation === In his essay '...,269
270,=== Tacit authoritarianism === The anarchist t...,270
271,== List of anarchist societies == * (Fejuv...,271


In [44]:
def filter_words(text):
    '''Given a string of text, tokenize the text and pull out only the nouns.'''
    is_noun = lambda pos: pos[:2] == 'NN' or pos[:2] == 'JJ' or pos[:2] == 'RB'
    tokenized = word_tokenize(text)
    all_nouns = [word for (word, pos) in pos_tag(tokenized) if is_noun(pos)] 
    return ' '.join(all_nouns)

In [60]:
fil_sent = df['sentences'].values
sen_list = []
for i in range(0,len(fil_sent)):
    b = filter_words(x[i])
    res = re.sub('['+string.punctuation+']', '', b).split() 
    listToStr = ' '.join([str(val) for val in res]) 
    sen_list.append(listToStr)
new_df = pd.DataFrame({'tokens': sen_list})
new_df

,tokens
0,Anarchism unjust replacement societies volunta...
1,institutions often several authors more specif...
2,Anarchism central disagreement other ideologie...
3,Anarchism usually much reflect
4,anarchism not fixed body doctrine single parti...
...,...
268,also sort court system still necessary crimes ...
269,Industrial civilisation essay Authority radica...
270,Tacit authoritarianism anarchist tendency othe...
271,List anarchist societies Fejuve CIPORFM Rojava...


In [61]:
df['tokens'] = new_df['tokens'].apply(word_tokenize)
df

,sentences,idx,tokens
0,'''Anarchism''' is an and that r...,0,"[Anarchism, unjust, replacement, societies, vo..."
1,"These institutions are often described as , a...",1,"[institutions, often, several, authors, more, ..."
2,Anarchism's central disagreement with other id...,2,"[Anarchism, central, disagreement, other, ideo..."
3,"Anarchism is usually placed on the of the ,...",3,"[Anarchism, usually, much, reflect]"
4,As anarchism does not offer a fixed body of do...,4,"[anarchism, not, fixed, body, doctrine, single..."
...,...,...,...
268,"It also states that ""some sort of 'court' syst...",268,"[also, sort, court, system, still, necessary, ..."
269,=== Industrial civilisation === In his essay '...,269,"[Industrial, civilisation, essay, Authority, r..."
270,=== Tacit authoritarianism === The anarchist t...,270,"[Tacit, authoritarianism, anarchist, tendency,..."
271,== List of anarchist societies == * (Fejuv...,271,"[List, anarchist, societies, Fejuve, CIPORFM, ..."


In [55]:
def nltk_stopwords():
    return set(nltk.corpus.stopwords.words('english'))

def prep_corpus(docs, additional_stopwords=set(), no_below=5, no_above=0.5):
  print('Building dictionary...')
  dictionary = Dictionary(docs)
  stopwords = nltk_stopwords().union(additional_stopwords)
  stopword_ids = map(dictionary.token2id.get, stopwords)
  dictionary.filter_tokens(stopword_ids)
  dictionary.compactify()
  dictionary.filter_extremes(no_below=no_below, no_above=no_above, keep_n=None)
  dictionary.compactify()

  print('Building corpus...')
  corpus = [dictionary.doc2bow(doc) for doc in docs]

  return dictionary, corpus

In [56]:
dictionary, corpus = prep_corpus(df['tokens'])
# dictionary, corpus = prep_corpus(res)

Building dictionary...
Building corpus...


In [57]:
MmCorpus.serialize('wiki_articles.mm', corpus)
dictionary.save('wiki_articles.dict')

In [58]:
%%time
lda = models.ldamodel.LdaModel(corpus=corpus, id2word=dictionary, num_topics=3, passes=50)
                                      
lda.save('anarchism.model')

CPU times: user 6.48 s, sys: 2.39 ms, total: 6.48 s
Wall time: 6.47 s


In [59]:
lda.show_topics(formatted=False)

[(0,
  [('anarchism', 0.23549642),
   ('anarchist', 0.05714904),
   ('also', 0.050938323),
   ('thought', 0.0412528),
   ('individualist', 0.037928566),
   ('political', 0.03703368),
   ('free', 0.034534976),
   ('important', 0.031042593),
   ('theory', 0.030835174),
   ('many', 0.024825606)]),
 (1,
  [('state', 0.1392098),
   ('society', 0.063399084),
   ('workers', 0.050940588),
   ('Anarchism', 0.049009014),
   ('Stirner', 0.048907403),
   ('education', 0.039513797),
   ('violence', 0.039005514),
   ('Anarchists', 0.03463155),
   ('Proudhon', 0.034123108),
   ('movements', 0.032882657)]),
 (2,
  [('anarchist', 0.12887174),
   ('anarchists', 0.106058225),
   ('movement', 0.05513517),
   ('social', 0.04442884),
   ('century', 0.037322965),
   ('ideas', 0.03348648),
   ('system', 0.029913664),
   ('first', 0.02771593),
   ('struggle', 0.026520567),
   ('human', 0.02649105)])]